## **pre-trained models for transfer learning**

In [2]:
import tensorflow_datasets as tfds
import tensorflow.keras as tfk

2024-01-30 08:25:31.633900: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 08:25:31.681062: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 08:25:31.681101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 08:25:31.682318: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 08:25:31.689827: I tensorflow/core/platform/cpu_feature_guar

In [3]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples
print(dataset_size)
class_names = info.features["label"].names
print(class_names)
n_classes = info.features["label"].num_classes
print(n_classes)

2024-01-30 08:25:33.719402: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 08:25:33.770364: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 08:25:33.773840: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

3670
['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
5


L355
2024-01-30 08:25:33.780791: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 08:25:33.783889: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 08:25:33.941167: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#

In [4]:
test_set_raw, val_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

batch_size = 32
preprocess = tfk.Sequential([
    tfk.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tfk.layers.Lambda(tfk.applications.xception.preprocess_input)
    ])
train_set = train_set_raw.map(lambda X, y: (preprocess(X), y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size).prefetch(1)
val_set = val_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)

data_augmentation = tfk.Sequential([
    tfk.layers.RandomFlip(mode="horizontal", seed=42),
    tfk.layers.RandomRotation(factor=0.05, seed=42),
    tfk.layers.RandomContrast(factor=0.2, seed=42)
    ])

2024-01-30 08:25:36.362631: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [5]:
base_model = tfk.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tfk.layers.GlobalAveragePooling2D()(base_model.output)
output = tfk.layers.Dense(n_classes, activation="softmax")(avg)
model = tfk.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

optimiser = tfk.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimiser,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=val_set, epochs=3)

Epoch 1/3


2024-01-30 08:25:42.667073: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-30 08:25:42.790875: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


 1/86 [..............................] - ETA: 9:30 - loss: 1.8210 - accuracy: 0.2188

2024-01-30 08:25:45.950207: I external/local_xla/xla/service/service.cc:168] XLA service 0x55921faa5e50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-30 08:25:45.950246: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1706603146.014078     742 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


86/86 [==============================] - 20s 162ms/step - loss: 0.6825 - accuracy: 0.8100 - val_loss: 0.5960 - val_accuracy: 0.8403
Epoch 2/3
86/86 [==============================] - 12s 139ms/step - loss: 0.2788 - accuracy: 0.9164 - val_loss: 0.5352 - val_accuracy: 0.8730
Epoch 3/3
86/86 [==============================] - 12s 139ms/step - loss: 0.1706 - accuracy: 0.9408 - val_loss: 0.5735 - val_accuracy: 0.8603


In [ ]:
for layer in base_model.layers[56:]:
    layer.trainable = True

optimiser = tfk.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimiser,
              metrics=["accuracy"])
history = model.fit(train_set, validation_data=val_set, epochs=10)

Epoch 1/10
86/86 [==============================] - 34s 284ms/step - loss: 0.2679 - accuracy: 0.9117 - val_loss: 0.5514 - val_accuracy: 0.8548
Epoch 2/10
86/86 [==============================] - 24s 274ms/step - loss: 0.0606 - accuracy: 0.9847 - val_loss: 0.3521 - val_accuracy: 0.9002
Epoch 3/10
86/86 [==============================] - 24s 275ms/step - loss: 0.0153 - accuracy: 0.9964 - val_loss: 0.3072 - val_accuracy: 0.9129
Epoch 4/10
86/86 [==============================] - 24s 275ms/step - loss: 0.0050 - accuracy: 0.9993 - val_loss: 0.3019 - val_accuracy: 0.9111
Epoch 5/10
86/86 [==============================] - 24s 276ms/step - loss: 0.0138 - accuracy: 0.9953 - val_loss: 0.3185 - val_accuracy: 0.9093
Epoch 6/10
86/86 [==============================] - 24s 276ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 0.3043 - val_accuracy: 0.9147
Epoch 7/10
86/86 [==============================] - 24s 277ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.3276 - val_accuracy: 0.9056